# 교차 검증(Cross validation)

 보통 train set으로 모델을 훈련, test set으로 모델을 검증한다. 여기에는 한 가지 약점이 존재한다. 고정된 test set을 통해 모델의 성능응ㄹ 검증하고 수정하는 과정을 반복하면, 결국 내가 만든 모델은 test set에만 잘 동작하는 모델이 된다.  
  
 즉, test set에 과적합(overfitting)하게 되므로, 다른 실제 데이터를 가져와도 예측을 수행하면 맞지 않은 경우가 자주 발생하게 된다. 이를 해결하고자 하는 것이 바로 교차 검증(Cross validation)이다. 

### 교차 검증 예제

In [3]:
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn import datasets 

In [4]:
# 데이터 로드
iris = datasets.load_iris()
X = iris.data
y = iris.target

X.shape, y.shape

((150, 4), (150,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [6]:
X_train.shape, y_train.shape

((90, 4), (90,))

# 교차 검증을 사용하지 않은 예제

In [7]:
from sklearn import svm

In [8]:
#모델 정의
clf = svm.SVC(kernel='linear', C=1)
#모델 학습
clf.fit(X_train, y_train)

SVC(C=1, kernel='linear')

In [9]:
# 평가
clf.score(X_test, y_test)

0.9666666666666667

# KFold

In [10]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [11]:
#모델 정의
clf = svm.SVC(kernel='linear', C=1)

In [12]:
#교차 검증
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [13]:
for train_index, valid_index in kf.split(X_train):
  print(f'train_index: {train_index} / valid_index: {valid_index}')
  break #모르는 것 생길 때 한 줄 찍어보고 너무 많을 때 break

train_index: [ 0  1  3  4  5  6  9 10 11 12 14 15 17 18 19 20 21 22 23 25 27 28 29 31
 32 34 35 36 37 38 39 40 42 44 46 47 48 49 50 51 52 54 55 57 58 59 60 61
 62 64 65 67 68 69 70 71 72 73 74 75 76 77 79 80 81 82 83 84 85 86 87 88] / valid_index: [ 2  7  8 13 16 24 26 30 33 41 43 45 53 56 63 66 78 89]


In [16]:
train_index

array([ 0,  1,  3,  4,  5,  6,  9, 10, 11, 12, 14, 15, 17, 18, 19, 20, 21,
       22, 23, 25, 27, 28, 29, 31, 32, 34, 35, 36, 37, 38, 39, 40, 42, 44,
       46, 47, 48, 49, 50, 51, 52, 54, 55, 57, 58, 59, 60, 61, 62, 64, 65,
       67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88])

In [14]:
X_train[train_index][:2]

array([[6. , 3.4, 4.5, 1.6],
       [4.8, 3.1, 1.6, 0.2]])

In [15]:
valid_x = X_train[valid_index]
valid_x.shape

(18, 4)

In [18]:
n_iter = 0
accuracy_lst = []

for train_index, valid_index in kf.split(X_train):
  n_iter += 1
  # 학습용, 검증용 데이터 구성성
  train_x, valid_x = X_train[train_index], X_train[valid_index]
  train_y, valid_y = y_train[train_index], y_train[valid_index]
  #학습
  clf.fit(train_x, train_y)
  #예측
  pred = clf.predict(valid_x)
  #평가
  accuracy = np.round(accuracy_score(valid_y, pred), 4)
  accuracy_lst.append(accuracy)
  print(f'{n_iter} 번째 Stratified Stratified K-Fold 정확도: {accuracy}, 학습데이터 label 분포: {train_x.shape}, 검증데이터 label 분포: {valid_x.shape}')

  # 최종 평가
print('-'*50)
print(f'교차 검증 정확도: {np.mean(accuracy_lst)} / 모델 평가: {clf.score(X_test, y_test)}')

1 번째 Stratified Stratified K-Fold 정확도: 1.0, 학습데이터 label 분포: (72, 4), 검증데이터 label 분포: (18, 4)
2 번째 Stratified Stratified K-Fold 정확도: 0.9444, 학습데이터 label 분포: (72, 4), 검증데이터 label 분포: (18, 4)
3 번째 Stratified Stratified K-Fold 정확도: 0.9444, 학습데이터 label 분포: (72, 4), 검증데이터 label 분포: (18, 4)
4 번째 Stratified Stratified K-Fold 정확도: 1.0, 학습데이터 label 분포: (72, 4), 검증데이터 label 분포: (18, 4)
5 번째 Stratified Stratified K-Fold 정확도: 1.0, 학습데이터 label 분포: (72, 4), 검증데이터 label 분포: (18, 4)
--------------------------------------------------
교차 검증 정확도: 0.97776 / 모델 평가: 0.9666666666666667


# cross val_predict()

만든 모델들을 통해 에측된 값들을 불러와 원하는 평가 계산 방법을 사용할 수 있도록 함

In [19]:
from sklearn.model_selection import cross_val_predict

#StrategyKflod

In [20]:
from sklearn.model_selection import StratifiedKFold

import pandas as pd


In [21]:
df_train = pd.DataFrame(data=X_train, columns=iris.feature_names)
df_train['label'] = y_train
df_train.shape 

(90, 5)

In [23]:
clf= svm.SVC(kernel='linear', C=1)

In [24]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [27]:
n_iter = 0
accuracy_lst = []

for train_index, valid_index in skf.split(df_train, df_train['label']):
  n_iter += 1
  # 학습용, 검증용 데이터 구성
  label_train = df_train['label'].iloc[train_index] #여기가 다름
  label_valid = df_train['label'].iloc[valid_index]

  train_x, valid_x = X_train[train_index], X_train[valid_index]
  train_y, valid_y = y_train[train_index], y_train[valid_index]
  # 학습
  clf.fit(train_x, train_y)
  # 예측
  pred = clf.predict(valid_x)
  # 평가
  accuracy = np.round(accuracy_score(valid_y, pred), 4)
  accuracy_lst.append(accuracy)
  print(f'{n_iter} 번째 Stratified Stratified K-Fold 정확도: {accuracy}, 학습데이터 label 분포: {label_train.value_counts()}, 검증데이터 label 분포: {label_valid.value_counts()}')

# 최종 평가
print('-'*50)
print(f'교차 검증 정확도: {np.mean(accuracy_lst)} / 모델 평가: {clf.score(X_test, y_test)}')

1 번째 Stratified Stratified K-Fold 정확도: 1.0, 학습데이터 label 분포: 0    27
2    24
1    21
Name: label, dtype: int64, 검증데이터 label 분포: 0    7
1    6
2    5
Name: label, dtype: int64
2 번째 Stratified Stratified K-Fold 정확도: 1.0, 학습데이터 label 분포: 0    28
2    23
1    21
Name: label, dtype: int64, 검증데이터 label 분포: 1    6
0    6
2    6
Name: label, dtype: int64
3 번째 Stratified Stratified K-Fold 정확도: 1.0, 학습데이터 label 분포: 0    27
2    23
1    22
Name: label, dtype: int64, 검증데이터 label 분포: 0    7
2    6
1    5
Name: label, dtype: int64
4 번째 Stratified Stratified K-Fold 정확도: 0.9444, 학습데이터 label 분포: 0    27
2    23
1    22
Name: label, dtype: int64, 검증데이터 label 분포: 0    7
2    6
1    5
Name: label, dtype: int64
5 번째 Stratified Stratified K-Fold 정확도: 1.0, 학습데이터 label 분포: 0    27
2    23
1    22
Name: label, dtype: int64, 검증데이터 label 분포: 0    7
2    6
1    5
Name: label, dtype: int64
--------------------------------------------------
교차 검증 정확도: 0.98888 / 모델 평가: 0.8833333333333333


## cross_val_predict
### 만든 모델을 통해 예측한 값들을 불러와 원하는 평가 계산 방법을 적용할 수 있도록 사용

In [28]:
from sklearn.model_selection import cross_val_predict

In [31]:
# 모델 정의
clf = svm.SVC(kernel='linear', C=1)
# 교차 검증
predicts = cross_val_predict(clf, X_train, y_train, cv=5)
print(f'각 검증 별 점수: \n{pd.Series(predicts)}')

각 검증 별 점수: 
0     1
1     0
2     2
3     1
4     1
     ..
85    0
86    2
87    1
88    2
89    0
Length: 90, dtype: int64


## cross_val_score
### 평가지표로 계산된 스코어에 대한 정보들을 확인하는 방법

In [25]:
from sklearn.model_selection import cross_val_score

In [26]:
# 모델 정의
clf = svm.SVC(kernel='linear', C=1)
# 교차 검증
scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=5)
print(f'각 검증 별 점수: \n{pd.Series(scores)}')
print(f'교차 검증 평균 점수: {np.mean(scores)}')

각 검증 별 점수: 
0    1.000000
1    1.000000
2    1.000000
3    1.000000
4    0.944444
dtype: float64
교차 검증 평균 점수: 0.9888888888888889


## cross_validate()
### 여러 개의 평가지표를 사용하고 싶을 때 사용

In [33]:
from sklearn.model_selection import cross_validate

In [34]:
# 모델 정의
clf = svm.SVC(kernel='linear', C=1)
# 교차 검증
result = cross_validate(clf, X_train, y_train, scoring='accuracy', cv=5, return_train_score=True)
print(f'각 검증 결과: \n: {pd.DataFrame(result)}')

각 검증 결과: 
:    fit_time  score_time  test_score  train_score
0  0.004496    0.000583    1.000000     0.986111
1  0.000929    0.000540    1.000000     1.000000
2  0.000885    0.000477    1.000000     0.986111
3  0.001101    0.000383    1.000000     0.986111
4  0.000735    0.000344    0.944444     1.000000
